<a href="https://colab.research.google.com/github/DaAnMaGi/PI_ML_OPS/blob/main/formulacion/formulacion_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descripción de la documentación.

En este documento se hace una revisión más exhaustiva de las bases de datos, y se realizan las transformaciones que se consideran más adecuadas para lograr la implementación de las funciones necesarias para el desarrollo de la API y el modelo de Machine Learning.

# Descarga de bibliotecas a utilizar

In [1]:
# Se descargan las bibliotecas necesarias
!pip install nltk
!pip install unidecode
!pip install langdetect
!pip install googletrans==4.0.0-rc1
!pip install inflect
!pip install regex
!pip install fuzzywuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=e894ef9c78935b1bbfb30db04035c2e4a571033b7aa7e68cc02a273fbe07ecdc
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6

In [2]:
# Se importan las bibliotecas a utilizar

# Pandas y numpy
import pandas as pd
import numpy as np

# Trabajo de archivos
import gzip
import json
from pandas import json_normalize

# Detección de idioma
from langdetect import detect, LangDetectException
from googletrans import Translator

#
import itertools

# Gráficación
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Análisis de lenguaje natural y texto
from unidecode import unidecode
import regex
import unicodedata
import inflect
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
from nltk.corpus import wordnet
from fuzzywuzzy import process

# Machine Learning
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
def guardar_archivo(DataFrame,RutaNuevoArchivo):
  ''' Convierte un Dataframe dado en json y lo comprime en formato gzip'''
  archivo_json = DataFrame.to_json()
  with gzip.open(RutaNuevoArchivo, "w") as f:
    f.write(archivo_json.encode('utf-8'))

# Se cargan las bases

In [64]:
# Se traen las rutas de los archivos a trabajar.
games_ruta = "/content/games_clean.json.gz"
genres_ruta = "/content/genres_games.json.gz"
specs_ruta = "/content/specs_games.json.gz"
#reviews_ruta = "/content/reviews_limpia.json.gz"
items_ruta = "/content/items_limpia.json.gz"


In [65]:
# Se convierten a pandas.

games = pd.read_json(games_ruta,compression="gzip")
genres = pd.read_json(genres_ruta,compression="gzip")
specs = pd.read_json(specs_ruta,compression="gzip")
#reviews = pd.read_json(reviews_ruta,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
items = pd.read_json(items_ruta,compression="gzip")

# Procesamiento del lenguaje natural en las reviews:

1.   Identificación de idioma de cada review.
2.   Separación de las reviews en inglés.
3.   Procesamiento de las reviews para obtenerlas lematizadas.



In [ ]:
reviews

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08
...,...,...,...,...,...
43971,wayfeng,730,True,its FUNNNNNNNN,2015-10-14
43972,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10
43973,72947282842,730,True,Prettyy Mad Game,2015-10-31
43974,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14


In [ ]:
# Se obtiene información sobre la distribución del dataset.
reviews["recommend"].value_counts(normalize=True)

True     0.906972
False    0.093028
Name: recommend, dtype: float64

In [ ]:
#Se crea una función para limpiar la base de datos, buscando mantener la escritura
# de los idiomas no alfabéticos

def clean_text(text):
    # Reemplazo de caracteres comunes
    text = re.sub(r"10/10","amazing game", text)
    text = re.sub(r"<3","loved the game", text)
    text = re.sub(r":\)","good game", text)
    # Eliminar caracteres numéricos y de puntuación
    text = regex.sub(r"[^\p{L}p{Z}]", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    # Dejar el texto en minusculas
    text = text.lower()
    # Normalizar el texto a la forma NFKC (compatibilidad y composición)
    text = unicodedata.normalize('NFKC', text)

    return text

# Se crea una segunda iteración de limpieza de texto para casos que no pudieron ser limpiados.
def clean_text2(text):
  text = re.sub("[^a-zA-Z]"," ",text)
  text = re.sub(r"\s{2,}", "", text)
  return text

In [ ]:
# Se limpia la base de datos.
reviews["review_clean"] = reviews["review"].apply(clean_text).replace([""," "],"no review")

In [ ]:
# Se crea una función para detectar el lenguaje.
# En caso de que no sea capaz de detectarlo, no trae ningún valor.
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException as e:
        return None

In [ ]:
# Se detecta el lenguaje para cada review
# reviews["language"] = reviews["review_clean"].apply(detect)
reviews["language"] = reviews["review_clean"].apply(detect_language)

In [ ]:
# Para los casos de lenguaje nulo se vuelve a aplicar una segunda capa de limpieza.
reviews.loc[reviews["language"].isna(), "review_clean"] = reviews.loc[reviews["language"].isna(), "review"].apply(clean_text2).replace(["", " "], "no review")

In [ ]:
# Se reemplaza el idioma de los nulos con "inglés"
reviews["language"].fillna("en",inplace=True)

In [ ]:
# Se obtiene la información de los lenguajes detectados en el dataframe.
reviews["language"].unique()

array(['en', 'tl', 'so', 'sv', 'af', 'cy', 'th', 'pl', 'pt', 'sl', 'fi',
       'ru', 'da', 'id', 'lt', 'it', 'no', 'sw', 'zh-cn', 'ko', 'nl',
       'fr', 'ro', 'es', 'hr', 'cs', 'hu', 'ca', 'de', 'et', 'tr', 'sk',
       'sq', 'lv', 'bg', 'vi', 'ja', 'uk', 'zh-tw', 'mk', 'ar'],
      dtype=object)

In [ ]:
# Se revisa la base obtenida.
reviews

,user_id,item_id,recommend,review,review_date,review_clean,language
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05,simple yet with great replayability in my opin...,en
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15,it s unique and worth a playthrough,en
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,great atmosphere the gunplay can be a bit chun...,en
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24,i know what you think when you see this title ...,en
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08,for a simple it s actually not all that simple...,en
...,...,...,...,...,...,...,...
43971,wayfeng,730,True,its FUNNNNNNNN,2015-10-14,its funnnnnnnn,no
43972,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10,awesome fantasy game if you don t mind the gra...,en
43973,72947282842,730,True,Prettyy Mad Game,2015-10-31,prettyy mad game,no
43974,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14,amazing game amazing game,tl


In [ ]:
reviews["language"].unique()

# Es importante notar que hay reviews (por ejemplo, del inglés y del español)
# que fueron detectadas en otros idiomas, esto debido a errores de escritura
# u ortografía.

array(['en', 'tl', 'so', 'sv', 'af', 'cy', 'th', 'pl', 'pt', 'sl', 'fi',
       'ru', 'da', 'id', 'lt', 'it', 'no', 'sw', 'zh-cn', 'ko', 'nl',
       'fr', 'ro', 'es', 'hr', 'cs', 'hu', 'ca', 'de', 'et', 'tr', 'sk',
       'sq', 'lv', 'bg', 'vi', 'ja', 'uk', 'zh-tw', 'mk', 'ar'],
      dtype=object)

In [ ]:
# Tras una revisión detalla, se detecta que la mayoría de las reviews de los siguientes
# idiomas estaban en inglés
lang_change = ["tl","so","sv","af","cy","sl","fi","id","lt","no","da",
               "sw","nl","ro","hr","hu","ca","et","lv","sq"]

# Idiomas que presentan una mezcla entre reviews en inglés y de otro idioma.
lang_mix = ["pl","it","de","tr","sk"]

# Se procede a realizar una revisión para los idiomas anteriormente detectados:
for row in range(reviews.shape[0]):
  # Se cambia el idioma de los lenguajes que se detectaron mayormente en inglés.
  if reviews.loc[row,"language"] in lang_change:
    reviews.loc[row,"language"] = "en"
  # Se intenta volver a revisar el lenguaje de los datos mezclados.
  elif reviews.loc[row,"language"] in lang_mix:
    reviews.loc[row,"language"] = detect(reviews.loc[row,"review_clean"])


In [ ]:
# Se decide guardar la base con los idiomas detectados.
guardar_archivo(reviews,"/content/reviews_lang.json.gz")

# Se decide seguir trabajando con la nueva base de reviews a partir del nuevo archivo
ruta_reviews_lang = "/content/reviews_lang.json.gz"
reviews = pd.read_json(ruta_reviews_lang,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")

In [ ]:
# Se crea una nueva columna en reviews para guardar las traducciones.
reviews["translation"] = ""

#Se procede a traducir cada uno de las reviews.
for row in range(reviews.shape[0]):
    # Si el texto está en inglés, no se traduce.
    if reviews.loc[row, "language"] == "en":
        reviews.loc[row, "translation"] = reviews.loc[row, "review_clean"]
    # Si el texto está en otro idioma, se busca traducir al inglés.
    else:
        try:
            translator = Translator()
            translation = translator.translate(reviews.loc[row, "review_clean"], dest="en")
            reviews.loc[row, "translation"] = translation.text
        except Exception as e:
            reviews.loc[row, "translation"] = "no review"

In [ ]:
# Se visualizan las traducciones de alguno de los idiomas.
reviews[reviews["language"] == "zh-cn"]

,user_id,item_id,recommend,review,review_date,review_clean,language,translation
223,2768820078,287700,True,一部小岛秀夫的游戏，游戏中居然还出现了这句话，算是konami对玩家的妥协么,2015-09-01,一部小岛秀夫的游戏 游戏中居然还出现了这句话 算是konami对玩家的妥协么,zh-cn,This sentence appeared in a game of Showfu Koh...
224,2768820078,377160,True,我想说，退款的都是sb,2015-11-18,我想说 退款的都是sb,zh-cn,I want to say that the refund is SB
225,2768820078,221380,True,"除了卖的太贵,士兵模型大多数国家都一样外，没有什么缺点",2015-08-08,除了卖的太贵 士兵模型大多数国家都一样外 没有什么缺点,zh-cn,There are no disadvantages except for being so...
227,2768820078,222420,False,如果拿来和ps2版的98um来比的话，阉割太严重，本来的3d背景居然又变回老98的2d背景了...,2015-06-12,如果拿来和ps 版的 um来比的话 阉割太严重 本来的 d背景居然又变回老 的 d背景了 语...,zh-cn,If it is comparable to the UM of the PS versio...
228,2768820078,271590,True,这代感觉恶搞的内容多了很多，外星人，僵尸，各种恶搞政府，种族歧视，资本家，环保，媒体的节目。...,2015-05-19,这代感觉恶搞的内容多了很多 外星人 僵尸 各种恶搞政府 种族歧视 资本家 环保 媒体的节目 ...,zh-cn,There are more content of spoofing this genera...
...,...,...,...,...,...,...,...,...
41865,76561198093386811,440,True,Steam上面貌似这个事最赚钱的游戏。但是这个是很好玩。,2014-06-17,steam上面貌似这个事最赚钱的游戏 但是这个是很好玩,zh-cn,no review
41868,76561198093386811,234310,False,Mac一次都没有成功打开过游戏。。。,2014-06-17,mac一次都没有成功打开过游戏,zh-cn,no review
42775,PulpCraft,369580,True,多么一个厚颜无耻的游戏……,2015-07-04,多么一个厚颜无耻的游戏,zh-cn,no review
43348,76561198122822618,271590,True,不得不说这款游戏绝对是一款吸引人的作品，虽然他两年前就在ps3和xb上有了，但丝毫也不影响我...,2015-04-23,不得不说这款游戏绝对是一款吸引人的作品 虽然他两年前就在ps 和xb上有了 但丝毫也不影响我...,zh-cn,no review


In [ ]:
# Tras la traducción se decide eliminar las columnas de "review" anteriores,
# dejando únicamente la traducción y la del lenguaje.
reviews_trans = reviews.drop(columns=["review", "review_clean", "language"]).rename(columns={"translation": "review"})

In [ ]:
reviews_trans

,user_id,item_id,recommend,review_date,review
0,76561197970982479,1250,True,2011-11-05,simple yet with great replayability in my opin...
1,76561197970982479,22200,True,2011-07-15,it s unique and worth a playthrough
2,76561197970982479,43110,True,2011-04-21,great atmosphere the gunplay can be a bit chun...
3,js41637,251610,True,2014-06-24,i know what you think when you see this title ...
4,js41637,227300,True,2013-09-08,for a simple it s actually not all that simple...
...,...,...,...,...,...
43971,wayfeng,730,True,2015-10-14,its funnnnnnnn
43972,76561198251004808,253980,True,2015-10-10,awesome fantasy game if you don t mind the gra...
43973,72947282842,730,True,2015-10-31,prettyy mad game
43974,ApxLGhost,730,True,2015-12-14,amazing game amazing game


In [ ]:
# Se guarda la nueva data en un nuevo documento:
# guardar_archivo(reviews_trans,"/content/reviews_en.json.gz")

# Se decide seguir trabajando con la nueva base de reviews a partir del nuevo archivo
ruta_reviews_en = "/content/reviews_en.json.gz"
reviews_en = pd.read_json(ruta_reviews_en,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")

In [ ]:
reviews_en

,user_id,item_id,recommend,review_date,review
0,76561197970982479,1250,True,2011-11-05,simple yet with great replayability in my opin...
1,76561197970982479,22200,True,2011-07-15,it s unique and worth a playthrough
2,76561197970982479,43110,True,2011-04-21,great atmosphere the gunplay can be a bit chun...
3,js41637,251610,True,2014-06-24,i know what you think when you see this title ...
4,js41637,227300,True,2013-09-08,for a simple it s actually not all that simple...
...,...,...,...,...,...
43971,wayfeng,730,True,2015-10-14,its funnnnnnnn
43972,76561198251004808,253980,True,2015-10-10,awesome fantasy game if you don t mind the gra...
43973,72947282842,730,True,2015-10-31,prettyy mad game
43974,ApxLGhost,730,True,2015-12-14,amazing game amazing game


In [ ]:
# Se obtiene información sobre la distribución del dataset.
reviews_en["recommend"].value_counts(normalize=True)

True     0.906972
False    0.093028
Name: recommend, dtype: float64

In [ ]:
# Se crea la lista de stopwords, como estamos trabajando con los reviews en inglés,
# trabajaremos con estos.
stopwords = nltk.corpus.stopwords.words('english')
stopwords = list(stopwords)

In [ ]:
# Se agregan a la lista de stopwords, después de una primera iteración palabras
# que después no representan información útil.
filtrar = []

if True:
  filtrar.append("game")
  filtrar.append("games")

In [ ]:
# se crea la siguiente función para obtener la categoría de la palabra que se usará
# para la lematización
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


In [ ]:
# Se limpia, tokeniza y lematiza cada una de las
# reviews de la base de datos.

reviews_lemma = []

for review in reviews_en.review:
  # Se limpia de caracteres no alfabéticos
  review = re.sub("[^a-zA-Z]"," ",str(review))
  # Se limpiza de caracteres repetidos numerosas veces.
  review = re.sub(r'(.)\1{2,}', r'\1\1', review)
  # Se tokeniza.
  review = nltk.word_tokenize(review)

  # Se lematizan las palabras
  review = [wnl.lemmatize(w, get_wordnet_pos(w)) for w in review]

  # Se eliminan palabras menores de 3 letras
  review = [word for word in review if len(word)>3]
  # Sacamos las Stopwords
  review = [word for word in review if word not in stopwords]
  review = [word for word in review if word not in filtrar]

  # Se unifica nuevamente la review
  review = " ".join(review)

  # Se agrega a la lista que luego servirá para agregar la info al dataframe
  reviews_lemma.append(review)


In [ ]:
# Se visualiza el proceso.
reviews_lemma[0:40]

['simple great replayability opinion zombie horde team work well left dead plus global leveling system alot earth zombie splatter whole family amaze sort rare',
 'unique worth playthrough',
 'great atmosphere gunplay chunky time definitely worth hope sequel sequel',
 'know think title barbie dreamhouse party intimidate title easily gotys cliche mechanic late simply good core gameplay noscope friend show dance move shame show true fashion color combination know easily range blast play',
 'simple actually simple truck drive simulator quite relax play simple easy basic wasd drive want much harder realistic manually change gear much harder turn reverse imagine would actual truck luckily reverse park extra point cause bloody hard suprisingly nice truck drive',
 'little play bore time passer recommend',
 'elegant integration gameplay story world development aesthetic',
 'random drop random quest stat point animation style reminiscent voodoo card',
 'balance tactic strategy potential reward b

In [ ]:
# Se agrega la lematización a el dataset.
reviews_en["review_lemma"] = reviews_lemma

In [ ]:
# Se visualiza el dataset
reviews_en

,user_id,item_id,recommend,review_date,review,review_lemma
0,76561197970982479,1250,True,2011-11-05,simple yet with great replayability in my opin...,simple great replayability opinion zombie hord...
1,76561197970982479,22200,True,2011-07-15,it s unique and worth a playthrough,unique worth playthrough
2,76561197970982479,43110,True,2011-04-21,great atmosphere the gunplay can be a bit chun...,great atmosphere gunplay chunky time definitel...
3,js41637,251610,True,2014-06-24,i know what you think when you see this title ...,know think title barbie dreamhouse party intim...
4,js41637,227300,True,2013-09-08,for a simple it s actually not all that simple...,simple actually simple truck drive simulator q...
...,...,...,...,...,...,...
43971,wayfeng,730,True,2015-10-14,its funnnnnnnn,funn
43972,76561198251004808,253980,True,2015-10-10,awesome fantasy game if you don t mind the gra...,awesome fantasy mind graphic recommend
43973,72947282842,730,True,2015-10-31,prettyy mad game,prettyy
43974,ApxLGhost,730,True,2015-12-14,amazing game amazing game,amaze amaze


In [ ]:
# Se procede a guardar la base con los datos lematizados para futuras iteraciones:
guardar_archivo(reviews_en,"/content/reviews_lemma.json.gz")

# Análisis de las reviews lematizadas.

In [78]:
ruta_reviews_lemma = "/content/reviews_lemma.json.gz"
reviews = pd.read_json(ruta_reviews_lemma,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
reviews

,user_id,item_id,recommend,review_date,review,review_lemma
0,76561197970982479,1250,True,2011-11-05,simple yet with great replayability in my opin...,simple great replayability opinion zombie hord...
1,76561197970982479,22200,True,2011-07-15,it s unique and worth a playthrough,unique worth playthrough
2,76561197970982479,43110,True,2011-04-21,great atmosphere the gunplay can be a bit chun...,great atmosphere gunplay chunky time definitel...
3,js41637,251610,True,2014-06-24,i know what you think when you see this title ...,know think title barbie dreamhouse party intim...
4,js41637,227300,True,2013-09-08,for a simple it s actually not all that simple...,simple actually simple truck drive simulator q...
...,...,...,...,...,...,...
43971,wayfeng,730,True,2015-10-14,its funnnnnnnn,funn
43972,76561198251004808,253980,True,2015-10-10,awesome fantasy game if you don t mind the gra...,awesome fantasy mind graphic recommend
43973,72947282842,730,True,2015-10-31,prettyy mad game,prettyy
43974,ApxLGhost,730,True,2015-12-14,amazing game amazing game,amaze amaze


In [79]:
# Se importa el modelo de análisis de sentimientos.
sia = SentimentIntensityAnalyzer()

In [80]:
# Se crea la definición del valor de respuesta que vamos a entender como
  # Positivo
pos = 0.5
  # y negativo
neg = -0.5

# Definimos una función para analizar el sentimiento de la review.
def sentiment_analisys(texto):
  sentimiento = sia.polarity_scores(texto)["compound"]

  # Se define la variable de respuesta.
  r = int

  # Si el sentimiento se define como positivo, nos marca 2.
  if sentimiento >= pos:
    r = 2
  # Si el sentimiento se define como negativo, nos marca 0.
  elif sentimiento <= neg:
    r = 0
  # En caso contrario se define como neutral en 1.
  else:
    r = 1

  return r

In [81]:
# Se hace el análisis de sentimiento de cada una de las reviews
reviews["sentiment_analysis"] = reviews['review'].apply(sentiment_analisys)
# Se hace el análisis de sentimiento con las reviews lemmatizadas
reviews["sentiment_lemma"] = reviews['review_lemma'].apply(sentiment_analisys)
# Se convierte el recommend en una nueva columna ("value_recommend"), donde se
# aplica 2 si es verdadero (para que tenga el mismo valor del análisis de
# sentimientos) y en 0 si es falso.
reviews["value_recommend"] = reviews["recommend"].apply(lambda x: 2 if x == True else 0)

In [82]:
# Con los tres valores anteriores, se intenta realizar una aproximación final al
# Sentimiento de la review.

def sentiment_final(val1,val2,val3):
  r = (val1 + val2 + val3)/3
  sentimiento = int
  # Si el valor es mayor a 1.5, se entenderá como un sentimiento positivo:
  if r > 1.5:
    sentimiento = 2
  # Si el valor es menor a 0.5, se entenderá como un sentimiento negativo:
  elif r < 0.5:
    sentimiento = 0
  # En caso contrario, se entenderá como un sentimiento neutro:
  else:
    sentimiento = 1

  return sentimiento

In [83]:
# Se crea una lista para guardar los resultados
sen_f = []

# Se analiza cada una de las filas en review
for i in range(reviews.shape[0]):
  val1 = reviews.loc[i,"sentiment_analysis"]
  val2 = reviews.loc[i,"sentiment_lemma"]
  val3 = reviews.loc[i,"value_recommend"]

  sent = sentiment_final(val1,val2,val3)

  sen_f.append(sent)

# Se agregan los resultados al dataframe
reviews["sentiment_final"] = sen_f

In [84]:
# Se visualiza el dataframe
reviews

,user_id,item_id,recommend,review_date,review,review_lemma,sentiment_analysis,sentiment_lemma,value_recommend,sentiment_final
0,76561197970982479,1250,True,2011-11-05,simple yet with great replayability in my opin...,simple great replayability opinion zombie hord...,2,2,2,2
1,76561197970982479,22200,True,2011-07-15,it s unique and worth a playthrough,unique worth playthrough,1,1,2,1
2,76561197970982479,43110,True,2011-04-21,great atmosphere the gunplay can be a bit chun...,great atmosphere gunplay chunky time definitel...,2,2,2,2
3,js41637,251610,True,2014-06-24,i know what you think when you see this title ...,know think title barbie dreamhouse party intim...,2,2,2,2
4,js41637,227300,True,2013-09-08,for a simple it s actually not all that simple...,simple actually simple truck drive simulator q...,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...
43971,wayfeng,730,True,2015-10-14,its funnnnnnnn,funn,1,1,2,1
43972,76561198251004808,253980,True,2015-10-10,awesome fantasy game if you don t mind the gra...,awesome fantasy mind graphic recommend,2,2,2,2
43973,72947282842,730,True,2015-10-31,prettyy mad game,prettyy,1,1,2,1
43974,ApxLGhost,730,True,2015-12-14,amazing game amazing game,amaze amaze,2,2,2,2


In [85]:
# Se revisan diferentes grupos (no review, y no recomendado)
# reviews[reviews["review"] == "no review"]
reviews[reviews["recommend"] == False]

,user_id,item_id,recommend,review_date,review,review_lemma,sentiment_analysis,sentiment_lemma,value_recommend,sentiment_final
37,76561198043472122,33440,False,2014-12-19,this game doesn t work,work,1,1,0,1
47,76561198066046412,359320,False,2015-12-28,charged me now its dollars got boring after a...,charge dollar boring hour,1,1,0,1
52,76561198070565427,570,False,2014-06-27,w,,1,1,0,1
65,boydeer,383080,False,2015-08-24,The game is very cool.,cool,1,1,0,1
90,sandwiches1,417860,False,2015-11-23,emily is a thot,emily thot,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
43849,Mooseox,417860,False,2015-12-14,i actuly liked emily brads a hes being nasty t...,actuly like emily brad nasty lovely girl could...,2,2,0,1
43864,pingaas,200510,False,2015-06-29,attention users with monitors should not purch...,attention user monitor purchase unless want pl...,1,1,0,1
43920,laislabonita75,305920,False,2015-06-13,i really didn t like it i m sorry slow and boring,really like sorry slow boring,1,1,0,1
43956,76561198222628548,370240,False,2015-09-28,this is just simply horrible i always lag and ...,simply horrible always always frozen horribly ...,0,0,0,0


In [86]:
# Se crea el dataframe de review final:

review_final = reviews.drop(columns=["review","review_lemma","sentiment_analysis",
                                    "sentiment_lemma","value_recommend"])
review_final.rename(columns={"sentiment_final":"sentiment_analysis"},inplace=True)

In [87]:
# Se agrega la fecha del año del review_date
review_final.insert(4,"review_year",review_final["review_date"].dt.year)

In [88]:
# Se elimina la columna "review_date" ya que resulta redundante.
review_final.drop(columns=["review_date"],inplace=True)

# Se cambia el nombre de "item_id" por "id_game"
review_final.rename(columns={"item_id":"id_game"},inplace=True)

In [89]:
# Se visualiza una última vez el dataframe
review_final

,user_id,id_game,recommend,review_year,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,1
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,2
...,...,...,...,...,...
43971,wayfeng,730,True,2015,1
43972,76561198251004808,253980,True,2015,2
43973,72947282842,730,True,2015,1
43974,ApxLGhost,730,True,2015,2


In [91]:
# Se guarda el dataframe final en un nuevo archivo.
guardar_archivo(review_final,"/content/reviews_sentiment.json.gz")

# Análisis games

In [7]:
games

,name,release_date,id_game,developer
0,Lost Summoner Kitty,2018-01-04,761140,Kotoshiro
1,Ironbound,2018-01-04,643980,Secret Level SRL
2,Real Pool 3D - Poolians,2017-07-24,670290,Poolians.com
3,弹炸人2222,2017-12-07,767400,彼岸领域
4,Log Challenge,None,773570,None
...,...,...,...,...
32128,Colony On Mars,2018-01-04,773640,"Nikita ""Ghost_RUS"""
32129,LOGistICAL: South Africa,2018-01-04,733530,Sacada
32130,Russian Roads,2018-01-04,610660,Laush Dmitriy Sergeevich
32131,EXIT 2 - Directions,2017-09-02,658870,"xropi,stev3ns"


In [8]:
# Se define una función para convertir diferentes formatos a fecha.
def convert_to_standard_date(date_str):
    try:
        # Intenta convertir formatos del estilo "feb 2018".
        if re.match(r'^[a-zA-Z]+ \d{4}$', date_str):
            return pd.to_datetime(date_str, format='%b %Y', errors='coerce')

        # Intenta convertir el formato del estilo "2016"
        elif re.match(r'^\d{4}$', date_str):
            return pd.to_datetime(date_str, format='%Y', errors='coerce')

        # Intenta convertir el formato "2019-05-10"
        else:
            return pd.to_datetime(date_str, errors='coerce')

    except Exception as e:
        # Maneja cualquier error que pueda ocurrir durante la conversión o formateo.
        return None

In [9]:
# Se transforman los datos a fechas válidas.
games["release_date"] = games["release_date"].astype(str).apply(convert_to_standard_date)
# Se obtiene el año en el que fue publicado
games["release_year"] = games["release_date"].dt.year

<ipython-input-8-df7b6b14d95c>:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_str, errors='coerce')


In [10]:
# Se elimina la columna "release_date" por redundancia
games.drop(columns=["release_date"],inplace=True)

In [55]:
# Se visualiza la data obtenida.
games

,name,id_game,developer,release_year
0,Lost Summoner Kitty,761140,Kotoshiro,2018.0
1,Ironbound,643980,Secret Level SRL,2018.0
2,Real Pool 3D - Poolians,670290,Poolians.com,2017.0
3,弹炸人2222,767400,彼岸领域,2017.0
4,Log Challenge,773570,None,NaN
...,...,...,...,...
32128,Colony On Mars,773640,"Nikita ""Ghost_RUS""",2018.0
32129,LOGistICAL: South Africa,733530,Sacada,2018.0
32130,Russian Roads,610660,Laush Dmitriy Sergeevich,2018.0
32131,EXIT 2 - Directions,658870,"xropi,stev3ns",2017.0


In [51]:
# Para los géneros, se especifica para aquellos con un género nulo como "Not specified"
genres["genre"].fillna("Not specified",inplace=True)

In [58]:
# Contar la cantidad de géneros por juego
conteo_generos = genres.groupby('id_game')['genre'].apply(list).reset_index(name='genre')

# Se combinan nuevamente los juegos los DataFrames usando merge
games_c = pd.merge(games, conteo_generos, on='id_game', how='left')

In [59]:
games_c

,name,id_game,developer,release_year,genre
0,Lost Summoner Kitty,761140,Kotoshiro,2018.0,"[Strategy, Action, Indie, Casual, Simulation]"
1,Ironbound,643980,Secret Level SRL,2018.0,"[Free to Play, Strategy, Indie, RPG, Card Game..."
2,Real Pool 3D - Poolians,670290,Poolians.com,2017.0,"[Free to Play, Simulation, Sports, Casual, Ind..."
3,弹炸人2222,767400,彼岸领域,2017.0,"[Action, Adventure, Casual]"
4,Log Challenge,773570,None,NaN,"[Action, Indie, Casual, Sports]"
...,...,...,...,...,...
32128,Colony On Mars,773640,"Nikita ""Ghost_RUS""",2018.0,"[Strategy, Indie, Casual, Simulation]"
32129,LOGistICAL: South Africa,733530,Sacada,2018.0,"[Strategy, Indie, Casual]"
32130,Russian Roads,610660,Laush Dmitriy Sergeevich,2018.0,"[Indie, Simulation, Racing]"
32131,EXIT 2 - Directions,658870,"xropi,stev3ns",2017.0,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe..."


In [63]:
# Se guarda la base de datos
guardar_archivo(games_c,"/content/games_genres_c.json.gz")

# Revisión de la base items

In [66]:
items

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...
5159890,76561198329548331,346330,BrainBread 2,0,0
5159891,76561198329548331,373330,All Is Dust,0,0
5159892,76561198329548331,388490,One Way To Die: Steam Edition,3,3
5159893,76561198329548331,521570,You Have 10 Seconds 2,4,4


In [67]:
# Se cambia el nombre de la columna para que sea la misma que en "games"
items_n = items.rename(columns={"item_id":"id_game"})
# Se elimina la columna "item_name" y la columna "playtime_2weeks" porque
# no proporciona información relevante
items_n.drop(columns=["playtime_2weeks","item_name"], inplace = True)

In [68]:
items_n

,user_id,id_game,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0
...,...,...,...
5159890,76561198329548331,346330,0
5159891,76561198329548331,373330,0
5159892,76561198329548331,388490,3
5159893,76561198329548331,521570,4


In [92]:
# Se buscarán eliminar todos los items_id que no se encuentren en la base de juegos.

# Se carga la base de juegos
games_c = pd.read_json("/content/games_genres_c.json.gz",compression="gzip")

# Se crea la lista de items a mantener.
keep = []

# Se obtienen todos los items (únicos)
items_check = items_n["id_game"].unique()

# Se hace la revisión de los items para identificar aquellos a mantener de
# aquellos que no.
for item in items_check.tolist():
  if item in games_c["id_game"]:
    keep.append(item)

# Se eliminan los items no identificados.
items_n = items_n[items_n["id_game"].isin(keep)]

In [74]:
items_n

,user_id,id_game,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0
...,...,...,...
5159685,ArkPlays7,730,4557
5159710,76561198326700687,11020,0
5159711,76561198326700687,6370,0
5159712,76561198326700687,13140,0


In [113]:
# Para obtener el año de las horas jugadas, se buscará el año de la review
# realizada por el usuario al juego, en caso de que no se tenga, se indicará
# para el año de lanzamiento del juego.

# Se carga el dataframe de reviews.
rev_f = pd.read_json("/content/reviews_sentiment.json.gz",compression="gzip")

# Se crea un nuevo dataframe trayendo las fechas que existen en reviews.
items_y = items_n.merge(rev_f,how="left",on=["user_id","id_game"]).drop(columns=["recommend","sentiment_analysis"])
items_y.rename(columns={"review_year":"year"},inplace=True)
# Y las fechas que queden en blanco se traen del dataframe de los juegos
items_y2 = items_y.merge(games_c[['id_game', 'release_year']],on="id_game", how="left")

# Se reemplazan los valores nulos
items_y2["year"].fillna(items_y2["release_year"],inplace=True)
# Se elimina "release_year" ya que resulta innecesaria
items_y2.drop(columns=["release_year"],inplace=True)

In [112]:
items_y2

,user_id,id_game,playtime_forever,year
0,76561197970982479,10,6,2000.0
1,76561197970982479,20,0,1999.0
2,76561197970982479,30,7,2003.0
3,76561197970982479,40,0,2001.0
4,76561197970982479,50,0,1999.0
...,...,...,...,...
1182216,ArkPlays7,730,4557,2012.0
1182217,76561198326700687,11020,0,2008.0
1182218,76561198326700687,6370,0,2011.0
1182219,76561198326700687,13140,0,NaN


In [75]:
# Se crea un nuevo dataframe con los datos de juegos que no tengan un
# playtime igual a 0, ya que no es un dato que
items_hours = items_n[items_n["playtime_forever"] != 0]
items_hours

,user_id,id_game,playtime_forever
0,76561197970982479,10,6
2,76561197970982479,30,7
8,76561197970982479,300,4733
9,76561197970982479,240,1853
10,76561197970982479,3830,333
...,...,...,...
5159325,76561198313357718,21100,115
5159331,76561198313357718,730,2317
5159332,POMFP0MF,730,2987
5159334,76561198314015545,220,64


# Otros

In [ ]:
# ejemplos de creación de las fórmulas

genero = "Indie"
year = 2018

juegos = []

for index, row in games_c.iterrows():
    for genre in row['genre']:
      if genre == genero:
        if row["release_year"] == year:
          juegos.append(row["name"])

print(f"Los juegos del género {genero} del año {year} son: {juegos}")

# Supongamos que tienes dos DataFrames: 'juegos' y 'generos'
juegos = pd.DataFrame({
    'id_juego': [1, 2, 3, 4],
   'nombre_juego': ['Juego A', 'Juego B', 'Juego C', 'Juego D']
})

generos = pd.DataFrame({
   'id_juego': [1, 1, 2, 3, 3, 3, 4],
    'genero': ['Acción', 'Aventura', 'Aventura', 'Estrategia', 'Simulación', 'Aventura', 'Acción']
})

# Contar la cantidad de géneros por juego
conteo_generos = generos.groupby('id_juego')['genero'].apply(list).reset_index(name='generos')

# Combinar los DataFrames usando merge
juegos_con_generos = pd.merge(juegos, conteo_generos, on='id_juego', how='left')



# Revisar función merge

# Supongamos que quieres contar la cantidad de juegos del género "Acción"
  genero_seleccionado = "Acción"

# Filtras el DataFrame para obtener solo las filas que contienen ese género
 juegos_con_genero_seleccionado = conteo_generos[conteo_generos['generos'].apply(lambda x: genero_seleccionado in x)]

# Obtienes la cantidad de juegos para ese género
  cantidad_juegos_accion = len(juegos_con_genero_seleccionado)

  print(f"La cantidad de juegos del género '{genero_seleccionado}' es: {cantidad_juegos_accion}")
